Problem 1-2

2. quantitative comparison via SequenceMatcher.ratio()

In [26]:
import difflib

In [45]:
# read files to strings
filename = " - FL_SYB_BetriebsaerztlicherDienst_ID8414.txt"

pdftotext = open('online pdftotext' + filename, 'r').read()
pdf2go = open('online pdf2go' + filename, 'r').read()
calibre = open('calibre' + filename, 'r').read()
pypdf2 = open('pypdf2' + filename, 'r').read()

In [52]:
baseline = difflib.SequenceMatcher(None, pdftotext, pdftotext).ratio()

pdf2go_ratio = difflib.SequenceMatcher(None, pdf2go, pdftotext).ratio()
calibre_ratio = difflib.SequenceMatcher(None, calibre, pdftotext).ratio()
pypdf2_ratio = difflib.SequenceMatcher(None, pypdf2, pdftotext).ratio()

print("baseline:", baseline)
print("pdf2go:", pdf2go_ratio)
print("calibre:", calibre_ratio)
print("pypdf2:", pypdf2_ratio)

baseline: 1.0
pdf2go: 0.45782220202938056
calibre: 0.35420936764594246
pypdf2: 0.3905145549897393


4. 
(i)

In [40]:
import os
import re

directory = "processed_txt/phone_numbers"
phone_numbers = r"\(?\d[\d\ \/\-\–)]{6,}\d"

file = open("extracted_phone_numbers.txt", "w")

for filename in os.listdir(directory):
  lines = open(os.path.join(directory, filename)).readlines()
  for line in lines:
    result = re.search(phone_numbers, line)
    if result:
        phone_number = result.group()
        phone_number = phone_number.replace(" ", "")
        phone_number = phone_number.replace("/", "")
        phone_number = phone_number.replace("-", "")
        phone_number = phone_number.replace("–", "")
        phone_number = phone_number.replace("(", "")
        phone_number = phone_number.replace(")", "")
        
        file.write(phone_number + "\n")

(ii)

(iii)

In [12]:
import os
import re

isbn_path = "processed_txt/isbn/gelbe_seiten_2019-43.txt"
isbn_numbers = r"^(?=(?:\D*\d){10}(?:(?:\D*\d){3})?$)(978|979)[\d-]+$"

file = open("extracted_isbn.txt", "w")

lines = open(isbn_path).readlines()
for line in lines:
    result = re.search(isbn_numbers, line)
    if result:
        file.write(result.group() + "\n")

##### 5.

In [3]:
import re
import os

directory = "processed_txt/scans"
phone_numbers = r"\(?\d[\d\ \/\-\–)]{6,}\d"

file = open("extracted_phone_numbers_scans.txt", "w")

for filename in os.listdir(directory):
    lines = open(os.path.join(directory, filename)).readlines()
    for line in lines:
        result = re.search(phone_numbers, line)
        if result:
            phone_number = result.group()
            phone_number = phone_number.replace(" ", "")
            phone_number = phone_number.replace("/", "")
            phone_number = phone_number.replace("-", "")
            phone_number = phone_number.replace("–", "")
            phone_number = phone_number.replace("(", "")
            phone_number = phone_number.replace(")", "")
            
            file.write(phone_number + "\n")

It works quite okay with the scanned and ocr interpreted text. But some phone numbers are missed, because they have wrong characters recognized in between them.
Examples:
- "(06221) 43.41 49-0" is not recognized because of the point in between numbers
- "(0 62 21] 4 18 55 58" is not recognized because the closing bracket is

The extraction can be changed to take these into account, but one never can be sure to get all misinterpretations right.

Furthermore, one difficulty is, that a lot of the phone numbers don't have a prefix, because e.g. it is a prefix for all numbers on one page. 